In [1]:
import pandas as pd
from sktime.utils.data_processing import from_multi_index_to_nested

In [2]:
data = pd.read_csv('most_0400_0600_1_5.csv', sep=';')

First, let's have an overview of our data.

In [3]:
data.describe()

,timestep_time,vehicle_angle,vehicle_pos,vehicle_slope,vehicle_speed,vehicle_x,vehicle_y,vehicle_z,person_angle,person_pos,person_slope,person_speed,person_x,person_y,person_z
count,2.067350e+06,1.228876e+06,1.228876e+06,1.228876e+06,1.228876e+06,1.228876e+06,1.228876e+06,1.228876e+06,838473.000000,837976.000000,838473.000000,837977.000000,838473.000000,838473.000000,838473.000000
mean,2.021132e+04,1.702282e+02,1.832031e+02,-6.081239e-01,5.878740e+00,4.394096e+03,2.036820e+03,9.745693e+01,161.218636,181.459266,-0.353450,7.413234,4313.035494,2069.419362,109.380626
std,9.779872e+02,9.915551e+01,5.781648e+02,5.173104e+00,6.913619e+00,1.581162e+03,1.031879e+03,1.157750e+02,100.731614,355.824890,6.752039,6.224017,1748.461993,1075.116513,120.228287
min,1.440000e+04,0.000000e+00,0.000000e+00,-8.873000e+01,0.000000e+00,-2.100000e-01,9.346000e+01,0.000000e+00,0.000000,0.000000,-90.000000,0.000000,-0.210000,93.460000,-0.080000
25%,1.952500e+04,8.179000e+01,1.553000e+01,-2.750000e+00,3.000000e-02,3.935520e+03,1.313255e+03,2.073000e+01,69.470000,13.380000,-2.250000,1.190000,3548.030000,1338.410000,26.250000
50%,2.035500e+04,1.968800e+02,4.360000e+01,0.000000e+00,2.180000e+00,4.513730e+03,1.859050e+03,6.108000e+01,157.550000,51.950000,0.000000,6.900000,4461.740000,1935.200000,67.800000
75%,2.102500e+04,2.323100e+02,1.335425e+02,1.090000e+00,1.233000e+01,5.155550e+03,2.665060e+03,1.069800e+02,240.100000,159.620000,1.530000,12.870000,5136.770000,2852.400000,126.250000
max,2.159500e+04,3.599900e+02,1.323421e+04,9.000000e+01,5.547000e+01,9.976570e+03,6.359290e+03,5.972100e+02,359.990000,4183.480000,90.000000,46.090000,9976.570000,6356.520000,575.200000


There are several columns that we won't need; for our problem, the relevant columns will be `vehicle_speed` and `vehicle_angle` (we're making an initial assumption that busses will have a more erratic trajectory than cars since they need to cover several different routes as opposed to private vehicles that usually go from one point to another). Let's drop the unused features.

In [4]:
# keeps only relevant columns
data = data[['vehicle_id', 'timestep_time', 'vehicle_speed', 'vehicle_angle', 'vehicle_type']]

Now let's see how the vehicle categories are defined.

In [5]:
data['vehicle_type'].unique()

array([nan, 'bus', 'train', 'hw_trailer', 'fastbicycle', 'motorcycle',
       'passenger4', 'passenger2b', 'truck', 'delivery', 'coach',
       'passenger2a', 'passenger3', 'passenger1', 'moped', 'trailer',
       'avgbicycle', 'hw_motorcycle', 'hw_passenger1', 'taxi',
       'emergency', 'slowbicycle', 'authority', 'uber', 'hw_coach',
       'hw_passenger2a', 'hw_truck', 'hw_passenger3', 'hw_passenger4',
       'hw_delivery', 'hw_passenger2b', 'army'], dtype=object)

According to the documentation, all the vehicle types that start with `passenger` are a subclass of cars. Therefore, we will unify these labels after we drop all categories that are neither cars nor busses.

In [6]:
# drops all entries that are neither cars nor busses
data = data[(data['vehicle_type'] == 'bus') | (data['vehicle_type'].str.startswith('passenger'))]

# unifies car label for all entries
data['vehicle_type'] = data['vehicle_type'].map(lambda x: 'car' if x.startswith('passenger') else x)

Now let's check if there are any null values on our modified dataset.

In [7]:
data.isna().sum()

vehicle_id       0
timestep_time    0
vehicle_speed    0
vehicle_angle    0
vehicle_type     0
dtype: int64

In order to work with the time-series algorithms from the `sktime` package, it is necessary to transform our dataset into a *nested* one. Since we cannot save a nested dataset, we will transform our dataframe into a multi-indexed one and save it to disk. Multi-indexed dataframes can be turned into nested by using the `from_multi_index_to_nested` method from the `sktime` package.

In [8]:
# multindex dataframe
data.sort_values(by=['vehicle_id', 'timestep_time'], inplace=True)
multiindex = pd.MultiIndex.from_frame(data[['vehicle_id', 'timestep_time']])
data.index = multiindex

# drops columns used in indexing
data.drop(['vehicle_id', 'timestep_time'], axis='columns', inplace=True)

In [9]:
data.to_csv('cleaned_data.csv')